# Workflow automation for a recommender engine using ALS Model in PySpark
### This notebook demonstrates the automation of a product recommendation engine given we have user-item interaction data in terms of the frequency of purchase for each unique user-item pair
### The automation is implemented by deploying the notebook as a job which reads the input data from a remote database, trains and runs the model and writes the output back to the designated database


## Importing the libraries and starting the Spark Session

In [4]:
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext 
from pyspark.sql.session import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
sc =SparkContext.getOrCreate()

In [6]:
spark = SparkSession(sc)

## Remote connection to the database ( Any supported database like DB2, Netezza, PDA )
### To enable the connection, add your database under the Data Sources tab in your project. You would need information about your database like JDBC URL, type, username/password

### Adding the asset (for example- the dataset which has the user-item interaction information) from the remote database after setting up the connection

In [7]:
# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.

from project_lib import Project
project = Project.access()
PDA_COPPEL_2020_credentials = project.get_connection(name="PDA_DIRECCIONRIESGOS")

import jaydebeapi, pandas as pd
PDA_COPPEL_2020_connection = jaydebeapi.connect('org.netezza.Driver',
    '{}://{}:{}/{}'.format('jdbc:netezza',
    PDA_COPPEL_2020_credentials['host'],
    PDA_COPPEL_2020_credentials['port'],
    PDA_COPPEL_2020_credentials['database']),
    [PDA_COPPEL_2020_credentials['username'],
    PDA_COPPEL_2020_credentials['password']])

#query = 'SELECT CLIENTECODIGO as ID_CTE, ADCLAFAM as ID_CLAS1, FRECUENCIA as FREQUENCY  FROM(SELECT *,TRIM(TO_CHAR(FAMILIA,"000")) AS FAM,TRIM(TO_CHAR(CLASE,"00")) AS CLAS,TRIM(TO_CHAR(AREA,"0")) AS AREA,(AREA||DEPARTAMENTO||CLAS||FAM) AS ADCLAFAM FROM(SELECT  CLIENTECODIGO,CLASE, FAMILIA, DEPARTAMENTO,SUM(CASE WHEN CLASE>"0" THEN 1 ELSE 0 END) AS FRECUENCIA,MAX(CASE WHEN CARTERA="Ropa" THEN 1  WHEN CARTERA="Muebles" THEN 2 WHEN CARTERA="Prestamos" THEN 3 ELSE 0 END) AS AREA FROM(SELECT *,CASE WHEN CLASE>"0" THEN 1 ELSE 0 END AS T_CLASE,CASE WHEN FAMILIA>"0" THEN 1 ELSE 0 END AS T_FAMILIA FROM DIRECCIONRIESGOS.ADMIN.TRANSACCIONESCARTERAS where FECHACORTE between "2017-01-31" and "2019-12-31" and CLIENTECODIGO not in (9001,9000) AND CLIENTECODIGO >10000) E GROUP BY CLIENTECODIGO, CLASE, FAMILIA, DEPARTAMENTO ORDER BY CLIENTECODIGO) T WHERE CLASE NOT IN (-99)) P ORDER BY CLIENTECODIGO,ADCLAFAM'
query = 'SELECT * FROM DIRECCIONRIESGOS.ADMIN.TRANSACCIONESCARTERAS limit 10'
data_df_1 = pd.read_sql(query, con=PDA_COPPEL_2020_connection)
data_df_1.head(10)

,CLIENTECODIGO,FECHACORTE,FECHACOMPRA,FACTURA,CODIGO,TIPOCOMPRA,CARTERA,CATEGORIA,DESCATEGORIA,DEPARTAMENTO,...,DESFAMILIA,CANTIDAD,PRECIO_VTA,DESCUENTO,ENGANCHE,INTERES,IMPORTE_IVA,PLAZOVENTA,NUM_TIE,NUM_CIUDAD
0,40655876,2020-01-31 00:00:00,2020-01-24 00:00:00,52377,317774,Credito,Ropa,11,Lencería,3,...,BABY DOLL,1,137.0,0.0,0.0,0.0,137.0,8,6537,38
1,9000,2020-01-31 00:00:00,2020-01-03 00:00:00,47196,180997,Contado,Ropa,3,Caballero Interior,1,...,BILLETERA,1,154.0,0.0,179.0,0.0,-25.0,0,1000,158
2,9000,2020-01-31 00:00:00,2020-01-02 00:00:00,262726,318894,Contado,Ropa,11,Lencería,3,...,CONJ BRASSIERE SEXY,1,137.0,0.0,198.0,0.0,-61.0,0,941,404
3,9000,2020-01-31 00:00:00,2020-01-30 00:00:00,668577,704056,Contado,Ropa,5,Cuidado Personal,7,...,BUCAL INFANTIL,1,22.0,0.0,0.0,0.0,22.0,0,373,46
4,9000,2020-01-31 00:00:00,2020-01-17 00:00:00,66304,145477,Contado,Ropa,4,Caballero Exterior,1,...,ML FSIA MODA,1,258.0,0.0,0.0,0.0,258.0,0,6599,71
5,45457860,2020-01-31 00:00:00,2020-01-02 00:00:00,272141,306364,Credito,Ropa,11,Lencería,3,...,CONJ BRASSIERE SEXY,1,146.0,0.0,0.0,0.0,146.0,8,393,1
6,8374783,2020-01-31 00:00:00,2020-01-21 00:00:00,497740,314297,Credito,Ropa,11,Lencería,3,...,PANTALETA,1,146.0,0.0,0.0,36.0,182.0,8,37,42
7,9000,2020-01-31 00:00:00,2020-01-13 00:00:00,104762,555366,Contado,Ropa,1,BEBES,5,...,CALCETA VESTIR NIÑA,1,77.0,0.0,0.0,0.0,77.0,0,1364,588
8,9000,2020-01-31 00:00:00,2020-01-25 00:00:00,167530,728005,Contado,Ropa,21,Impulso,7,...,BOLSA REGALO,1,16.0,0.0,128.0,0.0,-112.0,0,21,8
9,9000,2020-01-31 00:00:00,2020-01-10 00:00:00,522577,807564,Contado,Ropa,18,Deportivo,8,...,CORRER,1,344.0,0.0,488.0,0.0,-144.0,0,10,17


In [5]:

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.

from project_lib import Project
project = Project.access()
PDA_COPPEL_2020_credentials = project.get_connection(name="PDA_COPPEL_2020")

from pyspark.sql import SparkSession
sparkSession = SparkSession(spark).builder.getOrCreate()

df = sparkSession.read.format('jdbc') \
    .option('url', 'jdbc:netezza://{}:{}/{}'.format(PDA_COPPEL_2020_credentials['host'],PDA_COPPEL_2020_credentials['port'],PDA_COPPEL_2020_credentials['database'])) \
    .option('dbtable', 'USER_COPPEL_2020.MODELO_RECOMENDACION') \
    .option('user', PDA_COPPEL_2020_credentials['username']) \
    .option('password', PDA_COPPEL_2020_credentials['password']).load
df.show(5)



Missing personal credentials for "PDA_COPPEL_2020" (c28e7213-e2dc-42cc-aeb7-90cb4a526e0c).
Please configure them in the Watson Studio project.


KeyError: 'username'

In [8]:
import project_lib
project_lib.Project.get_file("TRANSACTIONS")

TypeError: get_file() missing 1 required positional argument: 'file_name'

In [12]:
#import dsx_core_utils, requests, os, io
#!pip install --proxy=https://10.33.128.80:8080 dsx
#!pip install --proxy=https://10.33.128.80:8080 dsx

#!pip install --proxy=https://10.33.128.80:8080 dsx
import dsx_core_utils
dataSet = dsx_core_utils.get_remote_data_set_info('TRANSACTIONS')
#final_stat = None
#dataSet = dsx_core_utils.get_remote_data_set_info('TRANSACTIONS')
#dataSource = dsx_core_utils.get_data_source_info(dataSet['datasource'])3
#sparkSession = SparkSession(sc).builder.getOrCreate()
# Load JDBC data to Spark dataframe
#dbTableOrQuery = '"' + (dataSet['schema'] + '"."' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table'] + '"'
#if (dataSet['query']):
#    dbTableOrQuery = "(" + dataSet['query'] + ") TBL"
#final_stat = sparkSession.read.format("jdbc").option("url", dataSource['URL']).option("dbtable", dbTableOrQuery).option("user",dataSource['user']).option("password",dataSource['password']).load()
#final_stat.show(5)

AttributeError: module 'dsx' has no attribute 'get_remote_data_set_info'

### Preparing data for the model

In [ ]:
ratings = (final_stat
    .select(
        'ID_CTE',
        'ID_CLAS1',
        'FREQUENCY',
    )
).cache()

### Make sure your data is 'integer' type 

In [ ]:
ratings.dtypes

In [ ]:
ratings.limit(10000)

### Spliting the data set to test and train for measuring the performance of the ALS Model

In [ ]:
(training, test) = ratings.randomSplit([0.8, 0.2])

## Build the recommendation model using ALS on the training data
### Collaborative filtering is commonly used for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix. spark.ml currently supports model-based collaborative filtering, in which users and products are described by a small set of latent factors that can be used to predict missing entries. spark.ml uses the alternating least squares (ALS) algorithm to learn these latent factors. 

### Parameters of ALS Model in PySpark realization are following:

##### NumBlocks is the number of blocks the users and items will be partitioned into in order to parallelize computation.
##### rank is the number of latent factors in the model.
##### maxIter is the maximum number of iterations to run.
##### regParam specifies the regularization parameter in ALS.
##### implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
##### alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0)

### Explicit vs. implicit feedback
#### The standard approach to matrix factorization based collaborative filtering treats the entries in the user-item matrix as explicit preferences given by the user to the item, for example, users giving ratings to products.
#### It is common in many real-world use cases to only have access to implicit feedback (e.g. views, clicks, purchases, likes, shares etc.). The approach used in spark.ml to deal with such data is taken from Collaborative Filtering for Implicit Feedback Datasets. Essentially, instead of trying to model the matrix of ratings directly, this approach treats the data as numbers representing the strength in observations of user actions (such as the number of clicks). Those numbers are then related to the level of confidence in observed user preferences, rather than explicit ratings given to items. The model then tries to find latent factors that can be used to predict the expected preference of a user for an item.

In [ ]:
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=2, regParam=0.01, 
          userCol="ID_CTE", itemCol="ID_CLAS1", ratingCol="FREQUENCY",
          coldStartStrategy="drop",
          implicitPrefs=True)

model = als.fit(ratings)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="FREQUENCY",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

###  Generate top k Item recommendations for each user
#### The value of 'k' here is 10 and can be changed by passing the desired value to the function



In [ ]:
userRecs = model.recommendForAllUsers(10)
userRecs.count()

#### Display the results : Each row represents the 'k' recommendations for each User

In [ ]:
userRecs.take(10)

#### For getting each recommendation as a row in the final csv, we break down the result generated above using the explode function

In [8]:
from pyspark.sql.functions import explode
#userRecs1=userRecs.withColumn("recommendations", explode(userRecs.recommendations))
#userRecs1.show()

####  Breaking down reach recommendation to separate columns

In [9]:
import select as s


In [10]:

userRecs1= userRecs1 \
  .select('ID_CTE', 'recommendations.*')    
   

NameError: name 'userRecs1' is not defined

### Display the results

In [ ]:
userRecs1.show() 

### Writing the Output back to the Remote Datasource
#### Thereby the output or resulting csv can be consumed directly by anyone who has the access to the remote database 

In [ ]:
new_table_name = 'RecommendationsResult'
userRecs1.coalesce(1).write \
   .format("jdbc") \
    .mode('overwrite') \
    .option("url", dataSource['URL']) \
    .option("dbtable", dataSet['schema']+"."+new_table_name) \
    .option("user", dataSource['user']) \
    .option("password", dataSource['password']) \
    .save()

## Saving the model for deployment in WML

In [11]:
from dsx_ml.ml import save


ModuleNotFoundError: No module named 'dsx_ml'

In [ ]:
type(model)

### Pack the model inside a pipeline 
#### Since the WML deployments allow saving Spark Pipelines directly, put the ALS model inside a Pipeline for direct deployment stage
#### Typically, A Pipeline is specified as a sequence of stages, and each stage is either a Transformer or an Estimator. These stages are run in order, and the input DataFrame is transformed as it passes through each stage. For this case, since the pipeline is bought in to action just for the sole cause of deployment, we do not use any transformers and such

In [12]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages=[model])

In [ ]:
model_alsWML = pipeline.fit(ratings)

In [ ]:
save(name = 'PySparkRecommenderPipeline',
     model = model_alsWML,
     test_data = ratings,
     algorithm_type = 'Classification',
     source='PySparkRecommenderWithWorkflow.ipynb',
     description='Recommender using PySpark')